In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import io
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import argparse
from apiclient.discovery import build
#from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
####Google Analytics Module
class GA(object): 
    def __init__(self, start_date, end_date, symbol):
        self.page_view = None
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
    def initialize_analyticsreporting(self):
        """
          Initializes the analyticsreporting service object.

      Returns:
        analytics an authorized analyticsreporting service object.
        """
        SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
        DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
        CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
      # Parse command-line arguments.
        parser = argparse.ArgumentParser(
          formatter_class=argparse.RawDescriptionHelpFormatter,
          parents=[tools.argparser])
        flags = parser.parse_args([])

      # Set up a Flow object to be used if we need to authenticate.
        flow = client.flow_from_clientsecrets(
          CLIENT_SECRETS_PATH, scope=SCOPES,
          message=tools.message_if_missing(CLIENT_SECRETS_PATH))

      # Prepare credentials, and authorize HTTP object with them.
      # If the credentials don't exist or are invalid run through the native client
      # flow. The Storage object will ensure that if successful the good
      # credentials will get written back to a file.
        storage = file.Storage('analyticsreporting.dat')
        credentials = storage.get()
        if credentials is None or credentials.invalid:
            credentials = tools.run_flow(flow, storage, flags)
        http = credentials.authorize(http=httplib2.Http())

      # Build the service object.
        analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)
        return analytics

    def get_report(self, analytics, symbol, col_name):
        VIEW_ID = '108608708'#barchart.com view_id
        #set filter pagePath according to different symbol.
        #total_filter = 'ga:pagePath=~(?i)/stocks/quotes/{}/*?'.format(symbol)
        #overview_filter = 'ga:pagePath=~(?i)/stocks/quotes/{}/overview/*?'.format(symbol)
        #build filter exrepression
        base = 'ga:pagePath=~(?i)/stocks/quotes/{}/*?'.format(symbol)
        filters = {}
        for sub_page in col_name:
            if sub_page == 'total_page_views':
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}.*$;ga:pagePath!~(?i)/story.*$;ga:pagePath!~(?i)/stocks/quotes/{}[a-z\.-].*$'.format(symbol, symbol)
            elif sub_page == 'overview':
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}/{}.*$,ga:pagePath=~(?i)/stocks/quotes/{}/$,ga:pagePath=~(?i)/stocks/quotes/{}$,ga:pagePath=~(?i)/stocks/quotes/{}\?search={}$;ga:pagePath!~(?i)/story.*$'.\
                format(symbol, sub_page, symbol, symbol, symbol, symbol)
            else:
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}/{}.*$;ga:pagePath!~(?i)/story.*$'.format(symbol, sub_page)
        final_response = {}
        trail = len(col_name)%5
        num = len(col_name) // 5
        for i in range(num):
            # Use the Analytics Service Object to query the Analytics Reporting API V4.
            response = analytics.reports().batchGet(
          body={
            'reportRequests': [
               {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 0]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 1]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 2]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 3]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 4]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                }
               ]
          }
      ).execute()
            if not final_response.get('reports'):
                final_response['reports'] = response['reports']
            else:
                final_response['reports'] += response['reports']
        for i in range(trail):
            trail_response = analytics.reports().batchGet(
                body = { 'reportRequests': [
                    {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[num*5 + i]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]
                   }
                ]
                }).execute()
            if not final_response.get('reports'):
                final_response['reports'] = trail_response['reports']
            else:
                final_response['reports'] += trail_response['reports']
        return final_response
    def json_cut(self, dic, col_name):
        temp_result = {}
        report = dic['reports']
        try:
            #get our date and pageview results
            for item in report[0].get('data', {}).get('rows', []):
                date = item['dimensions'][0][:4] +'-'+ item['dimensions'][0][4:6]+\
                                          '-'+item['dimensions'][0][6:]
                temp_result[date] = item['metrics'][0]['values']
            df = pd.DataFrame.from_dict(temp_result, orient='index')
            df = df.reset_index()
            df = df.rename(index=str, columns={'index': 'date', 0: col_name[0]})
            for i in range(1, len(report)):
                temp_result = {}
                for each in report[i].get('data', {}).get('rows', []):
                    date = each['dimensions'][0][:4] +'-'+ each['dimensions'][0][4:6]+\
                                          '-'+ each['dimensions'][0][6:]
                    temp_result[date] = each['metrics'][0]['values']
                temp_df = pd.DataFrame.from_dict(temp_result, orient = 'index')
                temp_df = temp_df.reset_index()
                temp_df = temp_df.rename(index = str, columns = {'index' : 'date', 0 : col_name[i]})
                df = pd.merge(df, temp_df, how = 'outer', on = 'date')
            df.insert(loc = 1, column = 'symbol', value = self.symbol)
            self.page_view = df
        except:
            return True
    def main(self, symbol):
        analytics = self.initialize_analyticsreporting()
        col_name = ['total_page_views', 'overview', 'performance', 'interactive-Chart', 'technical-chart', \
                   'technical-analysis', 'cheat-sheet', 'opinion' , 'trading-strategies', 'price-history', 'options', \
                    'volatility-greeks', 'covered-calls', 'naked-puts', 'option-spreads', 'news',\
                   'profile', 'sec-filings', 'competitors', 'comparison', 'earnings-estimates', \
                    'analyst-ratings', 'income-statement', 'cash-flow', 'balance-sheet']
        response = self.get_report(analytics, symbol, col_name)
        flag = self.json_cut(response, col_name)
        if flag:
            print(symbol + ' requests pageview failure!!!')   

In [15]:
#Get all stock symbols
df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
symbol_NYSE = list(df1.symbol)
symbol_NASDAQ = list(df2.symbol)
all_symbols = symbol_NYSE + symbol_NASDAQ
all_symbols = all_symbols

In [16]:
flag = 0
pageview_daily = dict()

In [19]:
to_drop_symbol = []
for i in range(0, len(all_symbols)//200 + 1):  # In order to aviod Google Analytics quota or limitation. I need to run by group.
    if i > 0:
        time.sleep(90) # Aviod quota limitation
    temp_symbols = all_symbols[i*200 : (i+1)*200]
    if i == 2:
        break
    flag = i
    print(datetime.datetime.now())
    for i, item in enumerate(temp_symbols):  
        #print('over here!!!') #test
        if i % 2 == 0:
            time.sleep(1)
        if i % 10 == 0:
            print(i)
        ga = GA('2018-01-01', '2018-12-31', item)
        ga.main(item)
        if ga.page_view is None:
            print(item + ' has no pageview data!!!')
            to_drop_symbol.append(item)
            continue
        if ga.page_view.empty:
            print(item + ' has no pageview data!!!')
            to_drop_symbol.append(item) #I need to drop the symbols if they have no pageview data.
            continue
        pageview_daily[item] = ga.page_view
    print(datetime.datetime.now())

2019-01-29 20:25:03.160575
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
2019-01-29 21:09:39.088715
2019-01-29 21:11:09.094652
0


KeyboardInterrupt: 

In [4]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Get_Summary(object):
    def __init__(self, start_date, end_date):
        #we need to catch error, if dates are illeagle
        self.start_date = start_date
        self.end_date = end_date
        self.page_views = dict()
        self.summary_res = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', \
                                          'Average Daily Page View', 'Std Daily Page View', 'Lowest Daily Page View', \
                                         '25% Daily Page View', '50% Daily Page View', '75% Daily Page View', \
                                         'Highest Daily Page View', 'Market_Cap(billion)'])
    def get_mkt_cap(self, symbol):

        url_shares = 'https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=OnDemand&symbols={}&fields=lastQtrEPS%2CannualEPS%2CttmEPS'.\
                format(symbol)
        try:
            fundmantal = pd.read_csv(url_shares)
            if fundmantal.empty:
                print(symbol + ' does not have public market value!!!')
                return None
            else:
                shares = fundmantal.loc[0, 'sharesOutstanding']/1000000 # Change to billion
        except HTTPError as he:
            print('getFinancialHighlights API Error!!! Run again')
        url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(symbol, self.end_date, self.end_date)
        try:
            price = pd.read_csv(url_price)
            if price.empty:
                print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date))
                mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
                return float(mkt['marketCapitalization'])/1000000
        except:
            print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(self.end_date))
            mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
            return float(mkt['marketCapitalization'])/1000000            
        price_end = price['close']
        return float(price_end)*shares
    def combine_page(self):
        df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
        df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
        symbol_NYSE = list(df1.symbol)
        symbol_NASDAQ = list(df2.symbol)
        all_symbols = symbol_NYSE + symbol_NASDAQ
        for item in all_symbols:
            try:
                temp = self.page_views[item]
            except:
                continue
            self.summary_res = pd.concat([self.summary_res, temp], sort = False)
    def get_data(self, item):
        try:
            pageview_daily = pd.read_csv('pageview_daily/pageview_daiyl_{}.csv'.format(item.lower()))
        except FileNotFound:
            return
        total = pageview_daily['total_page_views'].apply(int)
        total_pageview = sum(total)
        temp_res = pd.DataFrame(total.describe()).T
        temp_res = temp_res.reset_index()
        temp_res['startDate'] = ga.start_date
        temp_res['endDate'] = ga.end_date
        temp_res['total_page'] = total_pageview
        temp_res['symbol'] = ga.symbol
        temp_res['mkt(billion)'] = self.get_mkt_cap(item)
        temp_res = temp_res.drop(['index', 'count'], axis = 1)
        temp_res = temp_res[['symbol', 'startDate', 'endDate', 'total_page', 'mean', 'std', 'min', '25%', '50%', '75%',\
                            'max', 'mkt(billion)']]
        temp_res['mean'] = temp_res['mean'].round(2)
        temp_res['std'] = temp_res['std'].round(2)
        if temp_res['mkt(billion)'].isnull:
            pass
        else:
            temp_res['mkt(billion)'] = temp_res['mkt(billion)'].round(2)
        temp_res = temp_res.rename(index = str, columns = {'symbol' : 'Symbol', 'total_page' : 'Total Page View',\
                                                          'mean' : 'Average Daily Page View', \
                                                          'std' : 'Std Daily Page View', 'min' : 'Lowest Daily Page View',\
                                                          '25%' : '25% Daily Page View', '50%' : '50% Daily Page View', \
                                                          '75%' :  '75% Daily Page View', 'max' : 'Highest Daily Page View',\
                                                          'mkt(billion)' : 'Market_Cap(billion)'})
        self.page_views[item] = temp_res
    def main(self, symbols):
        """
        to_drop_symbol = self.get_page_view(symbols)
        #drop here
        for item in to_drop_symbol:
            symbols.remove(item)
        self.combine_page(symbols)        
        """
        print(datetime.datetime.now())
        pool = ThreadPool(30)
        pool.map(self.get_data, all_symbols) #important, no run except permission
        pool.close()
        pool.join()
        self.combine_page()
        print(datetime.datetime.now())

SyntaxError: 'continue' not properly in loop (<ipython-input-4-5c22962a50b5>, line 51)

In [ ]:
for i in range(0, len(all_symbols)//20 + 1):  # In order to aviod Google Analytics quota or limitation. I need to run by group.
    if i > 0:
        time.sleep(90) # Aviod quota limitation
    temp_symbols = all_symbols[i*20 : (i+1)*20]
    mytest = PAGE_VIEW('2018-01-01', '2018-12-31')
    to_drop_symbol = mytest.main(temp_symbols)
    ### prepare symbol pageView & Price Movement Dataset
    flag = i
    for each in temp_symbols:
        df = mytest.pageview_daily[each]
        df.fillna(0, inplace = True)
        idx = pd.date_range('2018-01-01', '2018-12-31')
        df.index = pd.DatetimeIndex(df['date'])
        df = df.reindex(idx, fill_value = 0)
        df.rename(index = str, columns = {'date' : 'to_drop'}, inplace = True)
        df.drop(['to_drop'], axis = 1, inplace = True)
        df.reset_index(inplace = True)
        df['index'] = pd.to_datetime(df['index']).dt.date
        df.rename(index = str, columns = {'index' : 'date'}, inplace = True)
        df.to_csv('pageview_daily_{}.csv'.format(each.lower()), index = False)
    mytest.summary_res.to_csv('summary_pageview_group{}.csv'.format(flag), index = False)

In [115]:
def temp_parallel(item):
    try:
        df = pd.read_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()))
    except FileNotFoundError:
        return
    df.fillna(0, inplace = True)
    idx = pd.date_range('2018-01-01', '2018-12-31')
    df.index = pd.DatetimeIndex(df['date'])
    df = df.reindex(idx, fill_value = 0)
    df.rename(index = str, columns = {'date' : 'to_drop'}, inplace = True)
    df.drop(['to_drop'], axis = 1, inplace = True)
    df.reset_index(inplace = True)
    df['index'] = pd.to_datetime(df['index']).dt.date
    df.rename(index = str, columns = {'index' : 'date'}, inplace = True)
    df['symbol'] = item
    df.to_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()), index = False)  

In [116]:
pool = ThreadPool(10)
#pool.map(temp_parallel, all_symbols)  important, no run except permission
pool.close()
pool.join()

In [3]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
pageview_price = Manager().dict()
def page_price_parallel(item):
    temp_pageview_price = pd.DataFrame([])
    try:
        pageview_daily = pd.read_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()))
    except FileNotFoundError:
        return 
    temp_pageview_price['Date'] = pageview_daily['date']
    temp_pageview_price['Symbol'] = item
    temp_pageview_price['PageView'] = pageview_daily['total_page_views'].apply(int)
    url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(item, '2018-01-01', '2018-12-31')
    try:
        whole_price = pd.read_csv(url_price)
    except EmptyDataError:
        print('{} has no price data during this time!!!'.format(item))
        whole_price = pd.DataFrame([], columns = ['tradingDay', 'close'])
    whole_price.rename(index = str, columns = {'tradingDay' : 'Date', 'close' : 'Close Price'}, inplace = True)
    temp_pageview_price = temp_pageview_price.merge(whole_price, how = 'outer', on = 'Date')
    price_col_name = list(whole_price.columns.values)
    name_to_drop = [n for n in price_col_name if n not in ['Date', 'Close Price']]
    temp_pageview_price.drop(name_to_drop, axis = 1, inplace = True)
    #drop observations for no trading.
    temp_pageview_price = temp_pageview_price[pd.notnull(temp_pageview_price['Close Price'])]
    temp = temp_pageview_price['PageView'] 
    temp_std = temp.rolling(100).std()
    temp_pageview_price['PageView_Change_in_Std_1'] = \
        ((temp.rolling(1).mean() - temp.rolling(100).mean().shift(1))/temp_std.shift(1)).round(2)
    temp_pageview_price['PageView_Change_in_Std_3'] = \
        ((temp.rolling(3).mean() - temp.rolling(100).mean().shift(3))/temp_std.shift(3)).round(2)
    temp_pageview_price['PageView_Change_in_Std_5'] = \
        ((temp.rolling(5).mean() - temp.rolling(100).mean().shift(5))/temp_std.shift(5)).round(2)
    temp_pageview_price['PageView_Change_in_Std_7'] = \
        ((temp.rolling(7).mean() - temp.rolling(100).mean().shift(7))/temp_std.shift(7)).round(2)
    temp_pageview_price['PageView_Change_in_Std_30'] = \
        ((temp.rolling(30).mean() - temp.rolling(100).mean().shift(30))/temp_std.shift(30)).round(2)
    temp_pageview_price['Close Price Change_Forward_1day'] = temp_pageview_price['Close Price'].diff()
    temp_pageview_price['Close Price Change_Forward_1day'] = temp_pageview_price['Close Price Change_Forward_1day'].shift(-1)
    temp_pageview_price['Close Price Percent Change_Forward_1day(%)'] = (temp_pageview_price['Close Price'].pct_change()*100).round(2)
    temp_pageview_price['Close Price Percent Change_Forward_1day(%)'] = temp_pageview_price['Close Price Percent Change_Forward_1day(%)'].shift(-1)
    temp_pageview_price['Close Price Change_Forward_3day'] = temp_pageview_price['Close Price'].diff(3)
    temp_pageview_price['Close Price Change_Forward_3day'] = temp_pageview_price['Close Price Change_Forward_3day'].shift(-3)
    temp_pageview_price['Close Price Percent Change_Forward_3day(%)'] = (temp_pageview_price['Close Price'].pct_change(3)*100).round(2)
    temp_pageview_price['Close Price Percent Change_Forward_3day(%)'] = temp_pageview_price['Close Price Percent Change_Forward_3day(%)'].shift(-3)
    temp_pageview_price['Close Price Change_Forward_5day'] = temp_pageview_price['Close Price'].diff(5)
    temp_pageview_price['Close Price Change_Forward_5day'] = temp_pageview_price['Close Price Change_Forward_5day'].shift(-5)
    temp_pageview_price['Close Price Percent Change_Forward_5day(%)'] = (temp_pageview_price['Close Price'].pct_change(5)*100).round(2)
    temp_pageview_price['Close Price Percent Change_Forward_5day(%)'] = temp_pageview_price['Close Price Percent Change_Forward_5day(%)'].shift(-5)
    temp_pageview_price['Close Price Change_Forward_7day'] = temp_pageview_price['Close Price'].diff(7)
    temp_pageview_price['Close Price Percent Change_Forward_7day(%)'] = (temp_pageview_price['Close Price'].pct_change(7)*100).round(2)
    temp_pageview_price['Close Price Change_Forward_7day'] = temp_pageview_price['Close Price Change_Forward_7day'].shift(-7)
    temp_pageview_price['Close Price Percent Change_Forward_7day(%)'] = temp_pageview_price['Close Price Percent Change_Forward_7day(%)'].shift(-7)
    temp_pageview_price['Close Price Change_Forward_30day'] = temp_pageview_price['Close Price'].diff(30)
    temp_pageview_price['Close Price Change_Forward_30day'] = temp_pageview_price['Close Price Change_Forward_30day'].shift(-30)
    temp_pageview_price['Close Price Percent Change_Forward_30day(%)'] = (temp_pageview_price['Close Price'].pct_change(30)*100).round(2)
    temp_pageview_price['Close Price Percent Change_Forward_30day(%)'] = temp_pageview_price['Close Price Percent Change_Forward_30day(%)'].shift(-30)
    pageview_price[item] = temp_pageview_price
    temp_pageview_price.to_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()), index = False)


In [3]:
#Get all stock symbols
df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
symbol_NYSE = list(df1.symbol)
symbol_NASDAQ = list(df2.symbol)
all_symbols = symbol_NYSE + symbol_NASDAQ
all_symbols = all_symbols

In [4]:
pool = ThreadPool(30)
pool.map(page_price_parallel, all_symbols)
pool.close()
pool.join()

BBCPW has no price data during this time!!!
ARDM has no price data during this time!!!
NAUH has no price data during this time!!!


In [143]:
#Combine the different group summary
final_summary = pd.DataFrame([])
for i in range(0, 306):
    temp_summary = pd.read_csv('raw_summary_data/summary_pageview_group{}.csv'.format(i))
    final_summary = pd.concat([final_summary, temp_summary])


In [144]:
final_summary['Market_Cap(billion)'] = final_summary['Market_Cap(billion)'].round(2)
final_summary.reset_index(inplace = True)
final_summary.drop(['index'], axis = 1, inplace = True)
final_summary.to_csv('summary_pageview_stocks.csv', index = False)

In [18]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Generate_Leaders():
    def __init__(self): 
        self.pageview_change = Manager().dict()
        self.my_date = None
    def get_mkt_cap(self, symbol, start_date, end_date):

        url_shares = 'https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=OnDemand&symbols={}&fields=lastQtrEPS%2CannualEPS%2CttmEPS'.\
                format(symbol)
        try:
            fundmantal = pd.read_csv(url_shares)
            if fundmantal.empty:
                print(symbol + ' does not have public market value!!!')
                return None
            else:
                shares = fundmantal.loc[0, 'sharesOutstanding']/1000000 # Change to billion
        except HTTPError as he:
            print('getFinancialHighlights API Error!!! Run again')
        url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(symbol, end_date, end_date)
        try:
            price = pd.read_csv(url_price)
            if price.empty:
                print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date, start_date))
                mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
                return float(mkt['marketCapitalization'])/1000000
        except:
            print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date, start_date))
            mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
            return float(mkt['marketCapitalization'])/1000000            
        price_end = price['close']
        return float(price_end)*shares
    def highest_pageview(self, start_date, end_date):
        df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
        df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
        symbol_NYSE = list(df1.symbol)
        symbol_NASDAQ = list(df2.symbol)
        all_symbols = symbol_NYSE + symbol_NASDAQ
        to_combine = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', 'Market_Cap(billion)'])
        for i, item in enumerate(all_symbols):
            try:
                pageview_daily = pd.read_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()))
            except FileNotFoundError:
                print('{} has no pageview data!!!'.format(item))
                continue
            pageview_daily['datetime'] =  pd.to_datetime(pageview_daily['date'])
            pageview_daily = pageview_daily.set_index(['datetime'])
            start_datetime = pd.to_datetime(start_date)
            end_datetime = pd.to_datetime(end_date)
            #print(start_date)
            pageview_daily = pageview_daily.loc[start_datetime : end_datetime]
            #print(pageview_daily)
            total = pageview_daily['total_page_views'].apply(int)
            total_pageview = sum(total)
            #temp_res = pd.DataFrame(total.describe()).T
            temp_res = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', 'Market_Cap(billion)'])
            #temp_res = temp_res.reset_index()
            temp_res.loc[0, 'Symbol'] = item
            temp_res['startDate'] = start_date
            temp_res['endDate'] = end_date
            temp_res['Total Page View'] = total_pageview
            temp_res['Market_Cap(billion)'] = self.get_mkt_cap(item, start_date, end_date)
            if temp_res['Market_Cap(billion)'].isnull().values.any():
                pass
            else:
                temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
            to_combine = pd.concat([to_combine, temp_res])
        #to_combine['Market_Cap(billion)'] = to_combine['Market_Cap(billion)'].round(2)
        to_combine.reset_index(inplace = True)
        to_combine.drop(['index'], axis = 1, inplace = True)
        to_combine.sort_values(by = ['Total Page View'], ascending = False, inplace = True)
        return to_combine
    def parallel(self, item):
        try:
            pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()))
        except FileNotFoundError:
            print('{} has no pageview data!!!'.format(item))
            return
        if pageview_price.empty:
            return
        #print(datetime.datetime.now())
        temp_res = pageview_price.set_index(['Date'])
        try:
            temp_res = temp_res.loc[[self.my_date]]
            temp_res.reset_index(col_fill = 'Date', inplace = True)
        except:
            print('{} has no trading on {}!!!'.format(item, self.my_date))
            return
        temp_res.loc[0, 'Market_Cap(billion)'] = self.get_mkt_cap(item, self.my_date, self.my_date)
        if temp_res['Market_Cap(billion)'].isnull().values.any():
            pass
        else:
            temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
        self.pageview_change[item] = temp_res 
    def highest_change(self, my_date):
        df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
        df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
        symbol_NYSE = list(df1.symbol)
        symbol_NASDAQ = list(df2.symbol)
        all_symbols = symbol_NYSE + symbol_NASDAQ
        all_symbols = all_symbols[:500]
        self.my_date = my_date
        print(datetime.datetime.now())
        pool = ThreadPool(50)
        pool.map(self.parallel, all_symbols)
        pool.close()
        pool.join()
        print(datetime.datetime.now())
        """

                for i, item in enumerate(all_symbols):
            if i % 500 == 0:
                print(i)
            self.parallel(item)
        """


In [19]:
leader = Generate_Leaders()
highest_change = leader.highest_change('2018-09-27')
#highest_change.to_csv('highest_change_time_window_{}.csv'.format(each), index = False)

2019-02-04 09:02:51.397140
ACA has no trading on 2018-09-27!!!
AQNA has no trading on 2018-09-27!!!
BC-B has no pageview data!!!
BCSF has no trading on 2018-09-27!!!
BC-A has no trading on 2018-09-27!!!
BHR-D has no trading on 2018-09-27!!!


Process ForkPoolWorker-58:
Process ForkPoolWorker-57:
Process ForkPoolWorker-60:
Process ForkPoolWorker-54:
Process ForkPoolWorker-53:
Process ForkPoolWorker-52:
Process ForkPoolWorker-40:
Process ForkPoolWorker-51:
Process ForkPoolWorker-37:
Process ForkPoolWorker-22:
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-77:
Process ForkPoolWorker-47:
Process ForkPoolWorker-71:
Process ForkPoolWorker-74:
Process ForkPoolWorker-61:
Process ForkPoolWorker-67:
Process ForkPoolWorker-75:
Process ForkPoolWorker-25:
Process ForkPoolWorker-24:
Process ForkPoolWorker-59:
Process ForkPoolWorker-78:
Process ForkPoolWorker-76:
Process ForkPoolWorker-62:
Process ForkPoolWorker-64:
Process ForkPoolWorker-65:
Pro

  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "

KeyboardInterrupt: 

  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "

  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multipr

  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()


In [12]:
leader.pageview_change['A']

,Date,Symbol,PageView,Close Price,PageView_Change_in_Std_1,PageView_Change_in_Std_3,PageView_Change_in_Std_5,PageView_Change_in_Std_7,PageView_Change_in_Std_30,Close Price Change_Forward_1day,Close Price Percent Change_Forward_1day(%),Close Price Change_Forward_3day,Close Price Percent Change_Forward_3day(%),Close Price Change_Forward_5day,Close Price Percent Change_Forward_5day(%),Close Price Change_Forward_7day,Close Price Percent Change_Forward_7day(%),Close Price Change_Forward_30day,Close Price Percent Change_Forward_30day(%),Market_Cap(billion)
0,2018-09-27,A,347,70.47617,-0.35,-0.48,-0.45,-0.41,-0.24,-0.25882,-0.37,1.027,1.46,0.00952,0.01,-1.14762,-1.63,-2.8534,-4.05,22.45


In [13]:
to_combine = pd.DataFrame([])

In [14]:
df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
symbol_NYSE = list(df1.symbol)
symbol_NASDAQ = list(df2.symbol)
all_symbols = symbol_NYSE + symbol_NASDAQ

In [15]:
a = 0
for item in all_symbols:
    a += 1
    if a % 100 == 0:
        print(a)
    try:
        temp = leader.pageview_change[item]
    except:
        continue
    to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100


In [17]:
to_combine.to_csv('Highest_change_total.csv', index = False)

In [18]:
columns = list(to_combine.columns)

In [21]:
columns

['Date',
 'Symbol',
 'PageView',
 'PageView_Std_100days',
 'PageView_Change_in_Std_1',
 'PageView_Change_in_Std_3',
 'PageView_Change_in_Std_5',
 'PageView_Change_in_Std_30',
 'Close Price',
 'Close Price Change_Forward_1day',
 'Close Price Percent Change_Forward_1day(%)',
 'Close Price Change_Forward_3day',
 'Close Price Percent Change_Forward_3day(%)',
 'Close Price Change_Forward_5day',
 'Close Price Percent Change_Forward_5day(%)',
 'Close Price Change_Forward_7day',
 'Close Price Percent Change_Forward_7day(%)',
 'Market_Cap(billion)']

In [22]:
highest_change_window1 = to_combine.drop(['PageView_Change_in_Std_3', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_7', 'PageView_Change_in_Std_30'], axis = 1)
highest_change_window1['Time_Window'] = 1
highest_change_window1.sort_values(by = ['PageView_Change_in_Std_1'], ascending = False, inplace = True)

In [23]:
highest_change_window3 = to_combine.drop(['PageView_Change_in_Std_1', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_7', 'PageView_Change_in_Std_30'], axis = 1)
highest_change_window3['Time_Window'] = 3
highest_change_window3.sort_values(by = ['PageView_Change_in_Std_3'], ascending = False, inplace = True)


In [24]:
highest_change_window5 = to_combine.drop(['PageView_Change_in_Std_1', 'PageView_Change_in_Std_3', 'PageView_Change_in_Std_7', 'PageView_Change_in_Std_30'], axis = 1)
highest_change_window5['Time_Window'] = 5
highest_change_window5.sort_values(by = ['PageView_Change_in_Std_5'], ascending = False, inplace = True)


In [25]:
highest_change_window7 = to_combine.drop(['PageView_Change_in_Std_1', 'PageView_Change_in_Std_3', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_30'], axis = 1)
highest_change_window7['Time_Window'] = 7
highest_change_window7.sort_values(by = ['PageView_Change_in_Std_7'], ascending = False, inplace = True)


In [26]:
highest_change_window30 = to_combine.drop(['PageView_Change_in_Std_1', 'PageView_Change_in_Std_3', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_7'], axis = 1)
highest_change_window30['Time_Window'] = 30
highest_change_window30.sort_values(by = ['PageView_Change_in_Std_30'], ascending = False, inplace = True)


In [27]:
highest_change_window1.to_csv('highest_change_window1.csv', index = False)
highest_change_window3.to_csv('highest_change_window3.csv', index = False)
highest_change_window5.to_csv('highest_change_window5.csv', index = False)
highest_change_window7.to_csv('highest_change_window7.csv', index = False)
highest_change_window30.to_csv('highest_change_window30.csv', index = False)

In [10]:
highest_pageview_3day = leader.highest_pageview('2018-12-01', '2018-12-03')
highest_pageview_5day = leader.highest_pageview('2018-12-01', '2018-12-05')

A has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AAC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AAN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AAP has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AAT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AB has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ABB has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ABBV has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ABC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ABEV has no trading price on 2018-12-05, the mkt_cap is obtained by getfi

AHT-H has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AHT-I has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AI has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIF has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIG has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIG.W has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIR has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AIV has no trading price on 2018-12-05, the mkt_cap is obtained b

APTV has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
APU has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
APY has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AQ has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AQN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AQNA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AQUA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AR has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ARA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ARC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ARCH has no trading price on 2018-12-05, the mkt_cap is obtained by g

AYX has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AZN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AZO has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AZRE has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AZUL has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
AZZ has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
B has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BABA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BAC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BAC-A has no trading price on 2018-12-05, the mkt_cap is obtained by g

BGX has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BGY has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BH has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BH.A has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHE has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHGE has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHK has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHLB has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHP has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
BHR has no trading price on 2018-12-05, the mkt_cap is obtained by g

...

THFF has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
THOR has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
THRM has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
THST has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TIGO has no pageview data!!!
TILE has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TIPT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TISA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TITN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TIVO has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TKKS has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TKKSU has no trading price on 2

TWMC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TWNK has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TWNKW has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TWOU has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TWST has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TXMD has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TXN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TXRH has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TYHT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TYME has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
TYPE has no trading price on 2018-12-05, the mkt_cap is obta

VECO has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VEON has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VERI has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VERU has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VGIT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VGLT has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VGSH has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VIA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VIAB has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VIAV has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
VICL has no trading price on 2018-12-05, the mkt_cap is obtai

WEYS has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHF has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHFBZ has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHLM has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHLR has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHLRD has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHLRP has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WHLRW has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WIFI has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WILC has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
WIN has no trading price on 2018-12-05, the mkt_cap is ob

ZAGG has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZBIO has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZBRA has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZEAL has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZEUS has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZFGN has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZG has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZGNX has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZION has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZIONW has no trading price on 2018-12-05, the mkt_cap is obtained by getfinancialhighlight API
ZIOP has no trading price on 2018-12-05, the mkt_cap is obtai

In [11]:
highest_pageview_30day = leader.highest_pageview('2018-12-01', '2018-12-30')

A has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AAC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AAN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AAP has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AAT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AB has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ABB has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ABBV has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ABC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ABEV has no trading price on 2018-12-30, the mkt_cap is obtained by getfi

AHT-H has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AHT-I has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AI has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIF has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIG has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIG.W has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIR has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AIV has no trading price on 2018-12-30, the mkt_cap is obtained b

APTV has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
APU has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
APY has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AQ has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AQN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AQNA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AQUA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AR has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ARA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ARC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ARCH has no trading price on 2018-12-30, the mkt_cap is obtained by g

AYX has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AZN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AZO has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AZRE has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AZUL has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
AZZ has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
B has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BABA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BAC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BAC-A has no trading price on 2018-12-30, the mkt_cap is obtained by g

BGX has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BGY has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BH has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BH.A has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHE has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHGE has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHK has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHLB has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHP has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
BHR has no trading price on 2018-12-30, the mkt_cap is obtained by g

...

THFF has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
THOR has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
THRM has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
THST has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TIGO has no pageview data!!!
TILE has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TIPT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TISA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TITN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TIVO has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TKKS has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TKKSU has no trading price on 2

TWMC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TWNK has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TWNKW has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TWOU has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TWST has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TXMD has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TXN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TXRH has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TYHT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TYME has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
TYPE has no trading price on 2018-12-30, the mkt_cap is obta

VECO has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VEON has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VERI has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VERU has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VGIT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VGLT has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VGSH has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VIA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VIAB has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VIAV has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
VICL has no trading price on 2018-12-30, the mkt_cap is obtai

WEYS has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHF has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHFBZ has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHLM has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHLR has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHLRD has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHLRP has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WHLRW has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WIFI has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WILC has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
WIN has no trading price on 2018-12-30, the mkt_cap is ob

ZAGG has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZBIO has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZBRA has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZEAL has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZEUS has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZFGN has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZG has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZGNX has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZION has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZIONW has no trading price on 2018-12-30, the mkt_cap is obtained by getfinancialhighlight API
ZIOP has no trading price on 2018-12-30, the mkt_cap is obtai

In [12]:
highest_pageview_3day.to_csv('highest_pageview_3day(12-01_12-03).csv', index = False)
highest_pageview_5day.to_csv('highest_pageview_5day(12-01_12-05).csv', index = False)
highest_pageview_30day.to_csv('highest_pageview_30day(12-01_12-30).csv', index = False)

In [24]:
df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
symbol_NYSE = list(df1.symbol)
symbol_NASDAQ = list(df2.symbol)
all_symbols = symbol_NYSE + symbol_NASDAQ  

In [25]:
all_symbols = all_symbols[:]

In [27]:
len(all_symbols)

6110

In [28]:
for each in all_symbols:
    #fix price change forward problem
    try:
        df = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(each.lower()))
    except:
        continue
    df.drop(['PageView_level_in_Std'], axis = 1, inplace = True) 
    temp = df['PageView']
    temp_res = ((temp - temp.rolling(100).mean())/temp.rolling(100).std()).round(2)
    df.insert(loc = 4, column = 'PageView_level_in_Std(rolling_100days)', value = temp_res)
    temp2 = df['PageView_level_in_Std(rolling_100days)']
    df.insert(loc = 5, column = 'PageView_3Days_in_Std', value = temp2.rolling(3).mean().round(2))
    df['PageView_5Days_in_Std'] = temp2.rolling(5).mean().round(2)
    df['PageView_30Days_in_Std'] = temp2.rolling(30).mean().round(2)
    df.insert(loc = 10, column = 'Close Price Change_Forward_3day', value = df['Close Price'].diff(3))
    df.insert(loc = 11, column = 'Close Price Percent Change_Forward_3day(%)', \
          value = (df['Close Price'].pct_change(3)*100).round(2))
    df['Close Price Change_Forward_3day'] = df['Close Price Change_Forward_3day'].shift(-3)
    df['Close Price Percent Change_Forward_3day(%)'] = df['Close Price Percent Change_Forward_3day(%)'].shift(-3)   
    df.to_csv('pageview_price/pageview_price_{}.csv'.format(each.lower()), index = False)

In [93]:
subpage = pd.read_csv('futures/subpage_summary_futures.csv')

In [95]:
subpage.to_csv('test.csv')

In [67]:
sub = subpage.drop(['date', 'total_page_views'], axis = 1)

In [68]:
sums = sub.select_dtypes(pd.np.number).sum().rename('total')
sub = sub.append(sums, sort = False)

In [69]:
sub.sort_values(by = 'total', axis = 1, ascending = False, inplace = True)

In [70]:
sub.drop(index = 'total', inplace = True)

In [71]:
subpage = subpage[['date', 'total_page_views']]

In [72]:
res = subpage.merge(sub, how = 'outer', left_index = True, right_index = True)

In [73]:
res.to_csv('futures/subpage_summary_futures.csv', index = False)